In [0]:
from pyspark.sql.functions import current_timestamp, lit
from pyspark.sql import functions as F
from datetime import datetime

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "postgres_public"
CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "bronze"
ORIGEM_DADOS = "supabase"

# Ingestao Landing → Bronze

## Proposito
Este notebook realiza a ingestao de dados da **Landing Zone** (postgres_public) para a **Camada Bronze**.

## Arquitetura de Dados

```
[Supabase PostgreSQL] 
       ↓
   [Fivetran]
       ↓
[Databricks Landing Zone: postgres_public]
       ↓
  **ESTE NOTEBOOK**
       ↓
[Camada Bronze: v_credit.bronze.*]
```

## O que este notebook faz:
1. **Le dados do schema Landing** (`v_credit.postgres_public`) sincronizado pelo Fivetran
2. **Adiciona colunas de metadata**:
   - `ingestion_timestamp`: Timestamp da carga
   - `origem`: Identificador da origem (sempre "supabase")
3. **Salva em formato Delta** na camada Bronze com propriedades otimizadas

## Decisoes Tecnicas:

### Por que adicionar metadata?
- **Rastreabilidade**: Saber quando cada registro foi carregado
- **Auditoria**: Identificar origem dos dados
- **Reprocessamento**: Facilita ingestao incremental futura

### Por que Full Overwrite?
- Bronze preserva historico completo da origem
- Simplifica logica de ingestao inicial
- Fivetran ja gerencia CDC na Landing

## Tabelas processadas:
- base_atendentes
- base_motivos
- canais
- chamados
- chamados_hora
- clientes
- custos
- pesquisa_satisfacao

## Dependencias:
- DDL Bronze executado (`ddl/bronze/ddl_tabelas_bronze.ipynb`)
- Fivetran sincronizando dados para `postgres_public`

## Proximo passo:
Executar notebooks Silver para limpar e validar os dados.

In [0]:
tabelas_postgres = {
    "base_atendentes": "base_atendentes",
    "base_motivos": "base_motivos",
    "canais": "canais",
    "chamados": "chamados",
    "chamados_hora": "chamados_hora",
    "clientes": "clientes",
    "custos": "custos",
    "pesquisa_satisfacao": "pesquisa_satisfacao"
}

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp

for tabela_origem, tabela_destino in tabelas_postgres.items():
    
    tabela_origem_completa = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{tabela_origem}"
    tabela_destino_completa = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{tabela_destino}"

    timestamp_carga = current_timestamp() 
    
    try:
        print(f"Iniciando ingestão da tabela: '{tabela_origem_completa}'")
        
        df = spark.read.table(tabela_origem_completa)
        
        df = df.withColumn("ingestion_timestamp", lit(timestamp_carga)) 
        df = df.withColumn("origem", lit(ORIGEM_DADOS))

        df.write.mode("append").saveAsTable(tabela_destino_completa)
        
        print(f"✅ Lote '{timestamp_carga}' da tabela '{tabela_destino_completa}' carregado com sucesso!")
        
    except Exception as erro:
        print(f"❌ Erro ao processar tabela '{tabela_destino_completa}': {str(erro)}\n")